In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.model_selection import cross_validate
import tensorflow as tf
from sklearn.metrics import mean_squared_error as MSE
from pathlib import Path
from sklearn.metrics import classification_report
import xgboost as xg 
from xgboost import XGBClassifier
import numpy as np
import pandas as pd 



In [2]:
pip install csv-to-sqlite

Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv_to_sqlite

In [4]:
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="UTF-8")
#change path to "Resources/insurance.csv" for jupyter notebook
input_files = ['Resources/insurance.csv']
#change path to "Resources/insurance.sqlite" for jupyter notebook
csv_to_sqlite.write_csv(input_files, "Resources/insurance.sqlite", options)


Error on table Resources/insurance.csv: 
 table [insurance] already exists
Written 0 rows into 1 tables in 0.014 seconds


0

In [9]:
import sqlalchemy
from sqlalchemy import create_engine, text, inspect, func
import warnings
engine = create_engine("sqlite:///Resources/insurance.sqlite")

#gives warning but easier to get the dataframe from querying
warnings.filterwarnings('ignore')
conn = engine.raw_connection()
insurance_df = pd.read_sql_query(sql=f"SELECT age ,sex, bmi, children, smoker , region , ROUND(charges,2) as charges FROM insurance", con=conn)

In [10]:
insurance_df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92
1,18,male,33.770,1,no,southeast,1725.55
2,28,male,33.000,3,no,southeast,4449.46
3,33,male,22.705,0,no,northwest,21984.47
4,32,male,28.880,0,no,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.55
1334,18,female,31.920,0,no,northeast,2205.98
1335,18,female,36.850,0,no,southeast,1629.83
1336,21,female,25.800,0,no,southwest,2007.95


In [11]:
insurance_df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [12]:
# Define features set
X = insurance_df.copy()
X.drop("charges", axis=1, inplace=True)
X.head()

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest


In [13]:
# Define target vector
y = insurance_df["charges"].values.reshape(-1, 1)
y[:5]

array([[16884.92],
       [ 1725.55],
       [ 4449.46],
       [21984.47],
       [ 3866.86]])

In [14]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(X)

In [15]:
X[:5]

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1,0,0,0,1
1,18,33.770,1,0,1,1,0,0,0,1,0
2,28,33.000,3,0,1,1,0,0,0,1,0
3,33,22.705,0,0,1,1,0,0,1,0,0
4,32,28.880,0,0,1,1,0,0,1,0,0


In [16]:
# Splitting 
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.3, random_state = 123) 

# XGboost

In [17]:

# Instantiation 


xgb_r  = GridSearchCV(
            estimator=xg.XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8, None],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4, 0.8]},
             scoring='neg_mean_absolute_error', verbose=3, n_jobs=-1)

grid_result = MultiOutputRegressor(xgb_r).fit(train_X, train_y)
best_params = grid_result.best_params_

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1;, score=-3277.868 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3;, score=-3532.270 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3;, score=-3669.150 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5;, score=-3676.478 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=7;, score=-3394.884 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=7;, score=-3664.572 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=4, min_child_weight=1;, score=-3452.491 total time

[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3;, score=-3400.347 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=3;, score=-3647.718 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=3;, score=-3816.538 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=3;, score=-3711.462 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=3;, score=-3885.064 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=3;, score=-3887.157 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=3;, score=-3945.185 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_

[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1;, score=-3526.159 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5;, score=-3534.205 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=7;, score=-3280.225 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=7;, score=-3510.605 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=4, min_child_weight=1;, score=-3762.499 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=4, min_child_weight=3;, score=-3358.977 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=4, min_child_weight=5;, score=-3727.318 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_

[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=7;, score=-3152.955 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=7;, score=-3196.965 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=7;, score=-3288.546 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=1;, score=-3543.619 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=1;, score=-3248.960 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=1;, score=-3254.687 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=1;, score=-3277.602 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.0, lear

[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=1;, score=-2880.890 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=3;, score=-2855.843 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=3;, score=-2841.089 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=3;, score=-2769.557 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=3;, score=-2732.243 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=3;, score=-2918.727 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=5;, score=-2838.194 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_

[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=3;, score=-3243.216 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=3;, score=-3211.857 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=3;, score=-3234.559 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=3;, score=-3220.943 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=5;, score=-3372.880 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=5;, score=-3201.446 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=5;, score=-3199.152 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_

[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=7;, score=-3030.513 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=7;, score=-2867.827 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=7;, score=-2860.856 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=7;, score=-2814.332 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=7;, score=-2878.865 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=5, min_child_weight=1;, score=-3232.036 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=5, min_child_weight=1;, score=-3102.858 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.

AttributeError: 'MultiOutputRegressor' object has no attribute 'best_params_'

In [19]:
xgb_r  = GridSearchCV(
            estimator=xg.XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "n_estimators" : (100, 200, 300, 500),
                        "max_depth": [ 3, 4, 5, 6, 8, None],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4, 0.8]},
             scoring='neg_mean_absolute_error', verbose=3, n_jobs=-1)

In [ ]:
xgb_r.fit(train_X, train_y)

Fitting 5 folds for each of 2592 candidates, totalling 12960 fits
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=1;, score=-3647.686 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=1;, score=-3345.074 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=1;, score=-3309.088 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=1;, score=-3352.586 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=1;, score=-3316.952 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=1;, score=-3262.446 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=1;, score=-3084.

[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=1;, score=-3451.547 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=1;, score=-3343.194 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=1;, score=-3306.681 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=1;, score=-3053.940 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7;, score=-3147.861 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7;, score=-3101.201 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7;, score=-2895.962 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate

[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=7;, score=-2729.480 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=3, min_child_weight=7;, score=-2851.324 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=1;, score=-3061.404 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=1;, score=-2987.943 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=1;, score=-2919.452 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=1;, score=-2790.182 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=5;, score=-3256.423 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_

[CV 1/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=7;, score=-2931.961 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=7;, score=-2981.056 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=7;, score=-2883.597 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=7;, score=-2723.533 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=None, min_child_weight=7;, score=-2762.491 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.1, max_depth=3, min_child_weight=1;, score=-2658.738 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.1, max_depth=3, min_child_weight=1;, score=-2686.637 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0

[CV 1/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.15, max_depth=6, min_child_weight=1;, score=-3129.229 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.15, max_depth=6, min_child_weight=1;, score=-3383.093 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.15, max_depth=6, min_child_weight=1;, score=-3130.885 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.15, max_depth=6, min_child_weight=1;, score=-3020.566 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.15, max_depth=6, min_child_weight=1;, score=-3067.977 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.15, max_depth=6, min_child_weight=3;, score=-3247.891 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.15, max_depth=6, min_child_weight=3;, score=-3150.011 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.1, learning_

...

[CV 3/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=200;, score=-2773.376 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2851.543 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2733.772 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2731.512 total time=   0.1s
[CV 5/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2843.843 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=100;, score=-3129.389 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=3, min

[CV 1/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=3, n_estimators=500;, score=-3820.784 total time=   0.1s
[CV 2/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=3, n_estimators=500;, score=-3558.199 total time=   0.1s
[CV 3/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=3, n_estimators=500;, score=-3551.213 total time=   0.1s
[CV 4/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=3, n_estimators=500;, score=-3420.504 total time=   0.1s
[CV 5/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=3, n_estimators=500;, score=-3356.113 total time=   0.1s
[CV 2/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=None, min_child_weight=5, n_estimators=500;, score=-3247.336 total time=   0.1s
[CV 3/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=Non

[CV 2/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=5, n_estimators=100;, score=-3142.049 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=5, n_estimators=100;, score=-3144.691 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=5, n_estimators=100;, score=-2950.556 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=5, n_estimators=100;, score=-3156.025 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=5, n_estimators=200;, score=-3563.535 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=5, n_estimators=200;, score=-3270.065 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_

[CV 2/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=3, n_estimators=500;, score=-3338.417 total time=   0.1s
[CV 3/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=3, n_estimators=500;, score=-3411.438 total time=   0.1s
[CV 4/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=3, n_estimators=500;, score=-3140.130 total time=   0.1s
[CV 5/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=3, n_estimators=500;, score=-3225.682 total time=   0.1s
[CV 4/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=5, n_estimators=200;, score=-3171.901 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=8, min_child_weight=5, n_estimators=200;, score=-3181.311 total time=   0.1s
[CV 1/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.05, max_depth=8, min

[CV 2/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=300;, score=-2982.233 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=300;, score=-3093.123 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=300;, score=-2896.137 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=300;, score=-2917.954 total time=   0.1s
[CV 1/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=500;, score=-3337.242 total time=   0.1s
[CV 5/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_weight=3, n_estimators=100;, score=-2915.363 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_

In [ ]:
xgb_r.best_score_

In [18]:
best_param = xgb_r.best_params_
best_param

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [33]:
# now we take the best possible params and build the model for training
xgboost_model = xgboost.XGBRegressor(n_estimators = best_param['n_estimators'],
                                     colsample_bytree=best_param['colsample_bytree'],
                                     learning_rate = best_param['learning_rate'],
                                     max_depth = best_param['max_depth'],
                                     min_child_weight=best_param['min_child_weight'])

In [34]:
# Fitting the model 
xgboost_model.fit(train_X, train_y) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [35]:
pred = xgb_r.predict(test_X)

In [36]:
# RMSE Computation 
rmse = np.sqrt(MSE(test_y, pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  4109.559429


In [41]:
xval_score = cross_validate(xgboost_model,
                            train_X, train_y,
                            scoring = ['r2', 'explained_variance'], verbose=3,
                            return_train_score=True, n_jobs=-1)
xval_score


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


{'fit_time': array([0.02609491, 0.02525902, 0.0187881 , 0.01402903, 0.01418304]),
 'score_time': array([0.00412107, 0.00279617, 0.00164413, 0.00124598, 0.0012219 ]),
 'test_r2': array([0.85788655, 0.78939986, 0.8490695 , 0.86501179, 0.85169603]),
 'train_r2': array([0.89208709, 0.90111757, 0.89534063, 0.89181671, 0.89389724]),
 'test_explained_variance': array([0.85791833, 0.78965494, 0.84951208, 0.86502817, 0.85170725]),
 'train_explained_variance': array([0.89208742, 0.9011179 , 0.89534077, 0.89181695, 0.89389748])}

[CV] END ...................... r2: (train=0.894, test=0.852) total time=   0.0s
[CV] END  explained_variance: (train=0.892, test=0.858) r2: (train=0.892, test=0.858) total time=   0.0s
[CV] END  explained_variance: (train=0.901, test=0.790) r2: (train=0.901, test=0.789) total time=   0.0s
[CV] END  explained_variance: (train=0.895, test=0.850) r2: (train=0.895, test=0.849) total time=   0.0s
[CV] END  explained_variance: (train=0.892, test=0.865) r2: (train=0.892, test=0.865) total time=   0.0s
[CV] END  explained_variance: (train=0.894, test=0.852) r2: (train=0.894, test=0.852) total time=   0.0s
